In [187]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from selenium import webdriver
from time import sleep
from selenium.webdriver.common.by import By
import dateutil
import sys
import logging
from typing import List
from google.cloud import bigquery
from google.cloud import bigquery
from google.oauth2 import service_account

In [188]:
#-------------------Extract-------------------------
urls = ["https://forecast.weather.gov/MapClick.php?lat=57.0826&lon=-135.2692#.Y-vs_9LMJkg",
        'https://forecast.weather.gov/MapClick.php?lat=45.5118&lon=-122.6756#.Y-vtHNLMJkg',
        ]
        
combined_df = pd.DataFrame()

for url in urls:
    r = requests.get(url)
    soup = BeautifulSoup(r.content,"html.parser")

    #various containers
    item1 = soup.find_all(id='current_conditions-summary')
    item2 = soup.find_all(id='current_conditions_detail')
    item4 = soup.find_all(id='tombstone-container')

    #raw data
    temp_f = [item.find(class_="myforecast-current-lrg").get_text() for item in item1]
    temp_min = soup.find('p', {'class': 'temp temp-low'}).text.strip()
    temp_max = soup.find('p', {'class': 'temp temp-high'}).text.strip()


    #df of temperatures
    df_temperature = pd.DataFrame({"temp" : temp_f,'tempmin': temp_min,'tempmax': temp_max})

    #df_2 is a df of current conditions in detail (Humidity, Wind Speed, Barometer, Dewpoint, Visibility, Last update)
    table = soup.find_all('table')
    df_2 = pd.read_html(str(table))[0]
    df_2 = df_2.pivot(columns=0, values=1).ffill().dropna().reset_index().drop(columns=['index'])

    #merge both dataframes
    temp_df=pd.concat([df_temperature,df_2],axis=1)

    #scrape lattitude, longitude, and elevation 
    lat_lon_elev = soup.find('span', {'class': 'smallTxt'}).text.strip()
    lat, lon, elev = re.findall(r'[-+]?\d*\.\d+|\d+', lat_lon_elev)

    #scrape name
    station = soup.find('h2', {'class': 'panel-title'}).text.strip()

    #add location, lat, long, and elev to source_df
    temp_df['elevation_ft'] = elev
    temp_df['latitude'] = lat
    temp_df['longitude'] = lon
    temp_df['weather_station'] = station

    combined_df = pd.concat([temp_df, combined_df], ignore_index=True, sort=False)

display(combined_df)



,temp,tempmin,tempmax,Barometer,Dewpoint,Humidity,Last update,Visibility,Wind Speed,elevation_ft,latitude,longitude,weather_station
0,48°F,Low: 32 °F,High: 49 °F,30.2 in (1022.69 mb),32°F (0°C),53%,16 Feb 04:35 PM PST,10.00 mi,S 3 MPH,20,45.59578,122.60917,"Portland, Portland International Airport (KPDX)"
1,38°F,Low: 28 °F,High: 35 °F,29.94 in (1013.9 mb),32°F (0°C),79%,16 Feb 2:53 pm AKST,6.00 mi,NW 3 mph,13,57.05,135.36,Sitka - Sitka Airport (PASI)


In [189]:
urls = ['https://www.localconditions.com/weather-portland-oregon/97201/past.php',
        "https://www.localconditions.com/weather-sitka-alaska/99835/past.php",
        ]

precip_df = pd.DataFrame()

for url in urls:
        r = requests.get(url)
        soup = BeautifulSoup(r.content,"html.parser")
        details = soup.select_one(".past_weather_express")
        data = details.find_all(text=True, recursive=False)
        data = [item.strip() for item in data]
        data = [item for item in data if item]
        data = data[2]
        df = pd.DataFrame([data], columns=['precip'])
        precip_df = pd.concat([precip_df, df], ignore_index=True, sort=False)
        
precip_df['precip'] = precip_df['precip'].str.extract(pat='(\d+\.?\d*)').astype(float)
precip_df = precip_df.fillna(0)

source_df =pd.concat([combined_df,precip_df],axis=1)
display(source_df)

/home/reed/Desktop/epicodus/team-week3/reed-work/venvteamweek/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  # This is added back by InteractiveShellApp.init_path()


,temp,tempmin,tempmax,Barometer,Dewpoint,Humidity,Last update,Visibility,Wind Speed,elevation_ft,latitude,longitude,weather_station,precip
0,48°F,Low: 32 °F,High: 49 °F,30.2 in (1022.69 mb),32°F (0°C),53%,16 Feb 04:35 PM PST,10.00 mi,S 3 MPH,20,45.59578,122.60917,"Portland, Portland International Airport (KPDX)",0.000
1,38°F,Low: 28 °F,High: 35 °F,29.94 in (1013.9 mb),32°F (0°C),79%,16 Feb 2:53 pm AKST,6.00 mi,NW 3 mph,13,57.05,135.36,Sitka - Sitka Airport (PASI),0.069


In [190]:
#-----------Data Transformations-----------------

# Convert 'lat' and 'lon' columns to float type
source_df['latitude'] = source_df['latitude'].astype(float)
source_df['longitude'] = source_df['longitude'].astype(float)

# Convert 'elev' column to int type
source_df['elevation_ft'] = source_df['elevation_ft'].astype(int)

# Extract the numeric part of the temperature string and convert it to int
source_df['temp'] = source_df['temp'].str.extract('(\d+)').astype(float)

# Extract the numeric part of the tempmin string and convert it to int
source_df['tempmin'] = source_df['tempmin'].str.extract('(\d+)').astype(float)

# Extract the numeric part of the temperature string and convert it to int
source_df['tempmax'] = source_df['tempmax'].str.extract('(\d+)').astype(float)

# Split wind speed values into components and convert speed to int type
source_df['Wind Speed'] = source_df['Wind Speed'].str.extract('(\d+)', expand=False).fillna(0).astype(float)

# Convert 'humidity' column to int type
source_df['Humidity'] = source_df['Humidity'].str.extract('(\d+)', expand=False).astype(float)

# Convert 'barometer' column to float type, and convert inches to millibars
source_df['Barometer'] = round(source_df['Barometer'].apply(lambda x: float(x.split()[0]) * 33.8639 if 'in' in x and x != 'NA' else None), 2)

# Convert 'Visibility' column to float type
source_df['Visibility'] = source_df['Visibility'].str.extract('(\d+\.\d+|\d+)', expand=False).astype(float).round(2)

#Convert 'last_update' column to UTC
source_df['Last update'] = source_df['Last update'].apply(lambda x: dateutil.parser.parse(x, tzinfos={"EST": -5 * 3600, "CST": -6 * 3600, "MST": -7 * 3600,"PST": -8 * 3600,"AKST": -9 * 3600,"HST": -10 * 3600}))
source_df['Last update'] = source_df['Last update'].apply(lambda x: x.astimezone(dateutil.tz.tzutc()))
source_df['datetime'] = source_df['Last update'].dt.strftime('%Y-%m-%d')
#source_df['datetime'] = pd.to_datetime(source_df['datetime'])

# make wind chill a float if exists and only display degree F
try:
    source_df[['Wind Chill']] = source_df['Wind Chill'].str.extract('(\d+)', expand=True).astype(float)
except:
    None

# extract the numeric value of dewpoint and only display the degree n farenheit
source_df[['Dewpoint']] = source_df['Dewpoint'].str.extract('(\d+)', expand=True).astype(float)

#change precip data type to float
source_df['precip'] = source_df['precip'].astype(float)

#rename weather station column to the city
def rename_station(value):
    if value == 'Portland, Portland International Airport (KPDX)':
        return 'Portland'
    elif value == 'Sitka - Sitka Airport (PASI)':
        return 'Sitka'

source_df['name'] = source_df['weather_station'].map(rename_station)

#change the names and order of columns to better fit the historical data
source_df = source_df.rename({'Humidity': 'humidity', 'Wind Speed': 'windspeed', 'Visibility': 'visibility','Wind Chill': 'windchill','Dewpoint':'dewpoint'}, axis=1) 
#this line only includes necesarry columns
source_df = source_df.reindex(['name','datetime','tempmax','tempmin','temp','windchill','dewpoint','humidity','precip','windspeed','visibility'], axis=1)
source_df = source_df.fillna(0)



display(source_df)

,name,datetime,tempmax,tempmin,temp,windchill,dewpoint,humidity,precip,windspeed,visibility
0,Portland,2023-02-17,49.0,32.0,48.0,0.0,32.0,53.0,0.000,3.0,10.0
1,Sitka,2023-02-16,35.0,28.0,38.0,0.0,32.0,79.0,0.069,3.0,6.0


In [191]:
#------------------load to BigQuery---------------------------------
#authorization
key_path = "/home/reed/.creds/dsa-deb-sa.json"
credentials = service_account.Credentials.from_service_account_file(
    key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"])

client = bigquery.Client(credentials=credentials, project=credentials.project_id,)

In [192]:
# **** SETUP LOGGING ****
# setup logging and logger
logging.basicConfig(            # setting up the root logger
    format='[%(levelname)-5s][%(asctime)s][%(module)s:%(lineno)04d] : %(message)s',
    level=logging.INFO,
    stream=sys.stdout
)
logger: logging.Logger = logging.getLogger('root')      # alias the root logger as `logger`
logger.setLevel(logging.DEBUG)                          # programmatically reassign the logging level

In [193]:
# change to match your filesystem
PROJECT_NAME = credentials.project_id
DATASET_NAME = "reed_weather_data"

# **** BIGQUERY DATASET CREATION ****

dataset_id = f"{PROJECT_NAME}.{DATASET_NAME}"
dataset = bigquery.Dataset(dataset_id)
dataset.location = "US"
dataset = client.create_dataset(dataset, exists_ok=True)

logger.info(f"Created weather dataset: {dataset.full_dataset_id}")

FACTS_TABLE_METADATA = {
    'weather_data': {
        'table_name': 'weather_data',
        'schema': [
            # indexes are written if only named in the schema
            bigquery.SchemaField('name', 'STRING', mode='REQUIRED'),
            bigquery.SchemaField('datetime', 'DATETIME', mode='NULLABLE'),
            bigquery.SchemaField('tempmax', 'FLOAT64', mode='NULLABLE'),
            bigquery.SchemaField('tempmin', 'FLOAT64', mode='NULLABLE'),
            bigquery.SchemaField('temp', 'FLOAT64', mode='NULLABLE'),
            bigquery.SchemaField('windchill', 'FLOAT64', mode='NULLABLE'),
            bigquery.SchemaField('dewpoint', 'FLOAT64', mode='NULLABLE'),
            bigquery.SchemaField('humidity', 'FLOAT64', mode='NULLABLE'),
            bigquery.SchemaField('precip', 'FLOAT64', mode='NULLABLE'),
            bigquery.SchemaField('windspeed', 'FLOAT64', mode='NULLABLE'),
            bigquery.SchemaField('visibility', 'FLOAT64', mode='NULLABLE'),
        ]
    }      
}

[INFO ][2023-02-16 16:56:34,421][3414794505:0012] : Created weather dataset: deb-01-372116:reed_weather_data


In [194]:
def load_table(
    df: pd.DataFrame, 
    client: bigquery.Client, 
    table_name: str, 
    schema: List[bigquery.SchemaField], 
    create_disposition: str = 'CREATE_IF_NEEDED', 
    write_disposition: str = 'WRITE_APPEND'
    ) -> None:
    """load dataframe into bigquery table

    Args:
        df (pd.DataFrame): dataframe to load
        client (bigquery.Client): bigquery client
        table_name (str): full table name including project and dataset id
        schema (List[bigquery.SchemaField]): table schema with data types
        create_disposition (str, optional): create table disposition. Defaults to 'CREATE_IF_NEEDED'.
        write_disposition (str, optional): overwrite table disposition. Defaults to 'WRITE_TRUNCATE'.
    """
    # *** run some checks ***
    # test table name to be full table name including project and dataset name. It must contain to dots
    assert len(table_name.split('.')) == 3, f"Table name must be a full bigquery table name including project and dataset id: '{table_name}'"
    # setup bigquery load job:
    #  create table if needed, replace rows, define the table schema
    job_config = bigquery.LoadJobConfig(
        create_disposition=create_disposition,
        write_disposition=write_disposition,
        schema=schema
    )
    logger.info(f"loading table: '{table_name}'")
    job = client.load_table_from_dataframe(df, destination=table_name, job_config=job_config)
    job.result()        # wait for the job to finish
    # get the resulting table
    table = client.get_table(table_name)
    logger.info(f"loaded {table.num_rows} rows into {table.full_table_id}")

In [195]:
# Load scraped Sitka and Portland daily weather to the historical data table

# get table name and schema from FACTS_TABLE_METADATA config param
table_name = f"{PROJECT_NAME}.{DATASET_NAME}.{FACTS_TABLE_METADATA['weather_data']['table_name']}"
schema = FACTS_TABLE_METADATA['weather_data']['schema']
# load dataframe
load_table(source_df, client, table_name, schema)

logger.info(f"loaded weather_data")

[INFO ][2023-02-16 16:56:34,560][1825632641:0029] : loading table: 'deb-01-372116.reed_weather_data.weather_data'


ArrowTypeError: object of type <class 'str'> cannot be converted to int

In [ ]:
#-------------------------Working Selinium code but could not get orking with airflow. an alternate method is being used---------------------
# URLS = ["https://www.wunderground.com/history/daily/us/or/portland",
#         'https://www.wunderground.com/history/daily/us/ak/sitka/PASI'
#         ]

# precip_df = pd.DataFrame()

# for URL in URLS:
    
#     #create selenium web driver
#     driver = webdriver.Chrome(r"C:/home/reed/.cache/selenium/chromedriver/linux64/109.0.5414.74")
#     driver.get(URL)
#     sleep(5)

#     soup = BeautifulSoup(driver.page_source, "html.parser")
    
#     #closes the pop up add if it appears otherwise continues to scrape the precip value from the page
#     try:
#         driver.find_element(By.XPATH,'/html/body/div[9]/div[3]/div/div/div/a').click()
#     except:
#         precip = driver.find_element(By.XPATH,"/html/body/app-root/app-history/one-column-layout/wu-header/sidenav/mat-sidenav-container/mat-sidenav-content/div/section/div[2]/div[1]/div[3]/div[1]/div/lib-city-history-summary/div/div[2]/table/tbody[2]/tr/td[1]").text

#     df = pd.DataFrame([precip], columns=['precip'])
#     precip_df = pd.concat([precip_df, df], ignore_index=True, sort=False)

#     #quits the selenium driver
#     driver.quit()

# source_df =pd.concat([combined_df,precip_df],axis=1)
# display(source_df)